In [1]:
import pickle
import pathlib
import os
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder


In [2]:
PROJECT_PATH = os.getcwd()
PROJECT_PATH = os.path.join(PROJECT_PATH, "..")
MODELS_DIR = pathlib.Path(PROJECT_PATH) / "store" / "models"

DATASET_DIR = pathlib.Path(PROJECT_PATH) / "data"

In [3]:
import pandas as pd

dataset = pd.read_csv(DATASET_DIR / "bank_marketing"/ "dataset.csv")
dataset

,age,job,marital,education,default,balance,housing,loan,contact,day_of_week,month,duration,campaign,pdays,previous,poutcome,y
0,27,management,single,secondary,no,35,no,no,cellular,4,jul,255,1,-1,0,NaN,no
1,54,blue-collar,married,primary,no,466,no,no,cellular,4,jul,297,1,-1,0,NaN,no
2,43,blue-collar,married,secondary,no,105,no,yes,cellular,4,jul,668,2,-1,0,NaN,no
3,31,technician,single,secondary,no,19,no,no,telephone,4,jul,65,2,-1,0,NaN,no
4,27,technician,single,secondary,no,126,yes,yes,cellular,4,jul,436,4,-1,0,NaN,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30902,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,NaN,yes
30903,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,NaN,yes
30904,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
30905,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,NaN,no


In [4]:
X, y = dataset.iloc[:, :-1], dataset.iloc[:, -1]

# Remove the bug in the dataset where the entire row has -9 values
mask = ~(X == -9).all(axis=1)
X = X[mask]
y = y[mask]

y = y.replace({"no": 0, "yes": 1}).astype(int)

/var/folders/r9/9n0wpy917zg1611cmsmqg0kr0000gn/T/ipykernel_42233/1292191429.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({"no": 0, "yes": 1}).astype(int)


In [5]:
def preprocess(X: pd.DataFrame):
    """
    The function will preprocess the data:
    1. Categorical features will be label encoded (Boy->1, Girl ->2)
    2. Numerical features will be scaled if the data is intended to be used for baseline. For cloud data set, no scaling will be preformed.

    Return pd.Dataframe
    """
    # Identify categorical and numeric columns
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    numeric_cols = X.select_dtypes(include=['number']).columns.tolist()

    # Initialize lists to store processed columns
    processed_columns = []

    # If there are categorical columns, apply one-hot encoding
    if categorical_cols:
        print("\nEncoding categorical columns...")
        # onehot_encoder = OneHotEncoder(categories='auto', sparse=False)
        # X_categorical = pd.DataFrame(onehot_encoder.fit_transform(X[categorical_cols]),
        #                              columns=onehot_encoder.get_feature_names_out(categorical_cols))
        label_encoder = LabelEncoder()
        X_categorical = pd.DataFrame()
        for col in categorical_cols:
            X_categorical[col] = label_encoder.fit_transform(X[col])

        processed_columns.append(X_categorical)

    # Apply standard scaling to the numeric columns
    if numeric_cols:
        print("\nScaling numerical columns...")
        scaler = MinMaxScaler()
        X_numeric = X[numeric_cols]
        # X_numeric = pd.DataFrame(scaler.fit_transform(X[numeric_cols]), columns=numeric_cols, index=X.index)

        processed_columns.append(X_numeric)

    # Combine the processed columns
    if processed_columns:
        X_processed = pd.concat(processed_columns, axis=1)
    else:
        X_processed = X.copy()  # If there are no categorical or numeric columns, keep the original dataframe


    return X_processed


X = preprocess(X)


Encoding categorical columns...

Scaling numerical columns...


In [6]:
X_sample, y_sample = X.iloc[2000:22000], y.iloc[2000:22000]
y_sample.value_counts()

y
0    18287
1     1713
Name: count, dtype: int64

In [7]:
X_test, y_test = X.iloc[:2000], y.iloc[:2000]
y_test.value_counts()

y
0    1878
1     122
Name: count, dtype: int64

In [12]:
import pandas as pd
import torch.nn as nn, tensorflow as tf
from keras.src.utils import to_categorical
from keras.src.layers import Dense, Dropout, Flatten, BatchNormalization
from keras.src.callbacks import EarlyStopping
from keras.src import Sequential


class DNNEmbedding(nn.Module):

    name = "dnn_embedding"

    def __init__(self, **kwargs):
        super(DNNEmbedding, self).__init__()

        X, y = kwargs.get("X"), kwargs.get("y")
        num_classes = len(set(y))
        y = to_categorical(y, num_classes=num_classes)

        model = Sequential()
        model.add(Dense(units=X.shape[1]//2, activation='tanh', name="embedding"))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
        model.add(Dense(units=num_classes, activation='softmax', name="output"))

        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        early_stop = EarlyStopping(patience=2, monitor="loss")

        model.fit(X, y,validation_data=[X_test, to_categorical(y_test,2)], epochs=50, batch_size=8, callbacks=[early_stop])
        self.model = model.layers[0]
        self.output_shape = (1, X.shape[1]//2)


    def forward(self, x):

        if type(x) is pd.DataFrame:
            x = x.to_numpy()

        embedding = self.model(x)
        return embedding


# Code to run on CPU
with tf.device('/CPU:0'):
    
    embedding = DNNEmbedding(X=X_sample, y=y_sample)



Epoch 1/50
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 271us/step - accuracy: 0.8278 - loss: 0.4450 - val_accuracy: 0.9390 - val_loss: 0.2306
Epoch 2/50
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 248us/step - accuracy: 0.9123 - loss: 0.2974 - val_accuracy: 0.9385 - val_loss: 0.2333
Epoch 3/50
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 247us/step - accuracy: 0.9163 - loss: 0.2813 - val_accuracy: 0.9385 - val_loss: 0.2209
Epoch 4/50
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 246us/step - accuracy: 0.9151 - loss: 0.2850 - val_accuracy: 0.9380 - val_loss: 0.2216
Epoch 5/50
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 245us/step - accuracy: 0.9170 - loss: 0.2745 - val_accuracy: 0.9385 - val_loss: 0.2132
Epoch 6/50
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 267us/step - accuracy: 0.9159 - loss: 0.2741 - val_accuracy: 0.9390 - val_loss: 0.1961
Epoch 7/50
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 245us/step - accuracy: 0.9168 - loss: 0.2643 - val_accuracy: 0.9390 - val_loss: 0.1848
Epoch 8/50
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 266us/step - accuracy: 0.9137 -

In [13]:
import numpy as np
from keras.src.layers import Input, Dense,  Flatten
from keras.src.layers import BatchNormalization, Activation, Conv2DTranspose
from keras.src.models import Model, Sequential
from keras.src.layers import LeakyReLU, Reshape, Conv2D, UpSampling2D, ReLU

class BaseEncryptor:

    name: str

    def __init__(self, input_shape=None, output_shape=None):
        self.model = None
        self.output_shape = output_shape
        self.input_shape = input_shape

    def build_generator(self, input_shape, output_shape):
        raise NotImplementedError("Subclasses should implement this method")

    def encode(self, inputs) -> np.array:
        inputs = np.expand_dims(inputs, axis=0)
        if self.model is None:
            input_shape = inputs.shape[1:]
            output_shape = self.output_shape or (1, inputs.shape[2])
            self.model = self.build_generator(input_shape, output_shape)
        return self.model(inputs).numpy()

class DCEncryptor(BaseEncryptor):

    name = "dc"
    
    def build_generator1(self, input_shape, output_shape):
        G = Sequential()
        G.add(Reshape(target_shape=[1, 1, self.input_shape], input_shape=[self.input_shape]))
        # No weights or activations here

        # 1x1x4096
        G.add(Conv2DTranspose(filters=64, kernel_size=4))
        G.add(Activation('relu'))
        # Weights index: 0, Activations index: 1

        # 4x4x64
        G.add(Conv2D(filters=64, kernel_size=4, padding='same'))
        G.add(BatchNormalization(momentum=0.7))
        G.add(Activation('relu'))
        # Weights index: 2, Activations index: 5
        G.add(UpSampling2D())
        # No weights or activations here

        # 8x8x64
        G.add(Conv2D(filters=32, kernel_size=4, padding='same'))
        G.add(BatchNormalization(momentum=0.7))
        G.add(Activation('relu'))
        # Weights index: 8, Activations index: 9
        G.add(UpSampling2D())
        # No weights or activations here

        # 16x16x32
        G.add(Conv2D(filters=16, kernel_size=4, padding='same'))
        G.add(BatchNormalization(momentum=0.7))
        G.add(Activation('relu'))
        # Weights index: 14, Activations index: 13
        G.add(UpSampling2D())
        # No weights or activations here

        # 32x32x16
        G.add(Conv2D(filters=8, kernel_size=4, padding='same'))
        G.add(BatchNormalization(momentum=0.7))
        G.add(Activation('relu'))
        # Weights index: 20, Activations index: 17
        G.add(UpSampling2D())
        # No weights or activations here

        # 64x64x8
        G.add(Conv2D(filters=4, kernel_size=4, padding='same'))
        G.add(BatchNormalization(momentum=0.7))
        G.add(Activation('relu'))
        # Weights index: 26, Activations index: 21
        G.add(UpSampling2D())
        # No weights or activations here

        # 128x128x4
        G.add(Conv2D(filters=3, kernel_size=4, padding='same'))
        G.add(Activation('sigmoid'))
        
        return G
        
    def build_generator(self, input_shape, output_shape):

        input_layer = Input(shape=input_shape)
        x = Flatten()(input_layer)
    
        x = Dense(4*4*256, use_bias=False)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)
    
        x = Reshape((4, 4, 256))(x)
    
        x = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
    
        x = Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
    
        x = Conv2DTranspose(32, (4, 4), strides=(2, 2), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
    
        output_image = Conv2DTranspose(3, (4, 4), strides=(1, 1), padding='same', use_bias=False, activation='sigmoid')(x)
    
        return Model(inputs=input_layer, outputs=output_image)
        
    def build_generator_vgg224(self, input_shape, output_shape):

        input_layer = Input(shape=input_shape)
        x = Flatten()(input_layer)

        x = Dense(7*7*256, use_bias=False)(x)
        x = BatchNormalization()(x)
        x = LeakyReLU()(x)

        x = Reshape((7, 7, 256))(x)
        x = Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

        x = Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)

        x = Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

        x = Conv2DTranspose(16, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

        output_image = Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')(x)

        return Model(inputs=input_layer, outputs=output_image)
    
    
encoder = DCEncryptor(output_shape=(1, 32, 32, 4))

In [16]:
from keras.api.applications import ResNet152V2, VGG16, EfficientNetB7
# from keras.api.applications.vgg16 import preprocess_input
from keras.api.applications.resnet_v2 import preprocess_input
import tensorflow as tf
from keras.api.models import load_model


def pad(tensor, original, target=600):
    pad_height = (target - original) // 2
    pad_width = (target - original) // 2
    padded_tensor = tf.pad(tensor, [[pad_height, pad_height], [pad_width, pad_width], [0, 0]], mode='CONSTANT', constant_values=0)

    # If the dimensions are odd, add an extra row/column to one side
    if (600 - 224) % 2 != 0:
        padded_tensor = tf.pad(padded_tensor, [[0, 1], [0, 1], [0, 0]], mode='CONSTANT', constant_values=0)

    return padded_tensor[np.newaxis, ...]

def preprocess_image(image):
    # Assuming 'image' is your input tensor
    resized_image = tf.image.resize(image, (32, 32))
    return resized_image


class VGG16CloudModel:
    name = "vgg16"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.model = self.get_model()
        self.input_shape = (32,32,3)#(224, 224, 3)
        self.output_shape = (1,1000)

    def fit(self, X_train, y_train, **kwargs):
        pass

    def get_model(self):
        # Load the pretrained VGG16 model with ImageNet weights
        model = load_model("/Users/eden.yavin/Projects/Tabular-Cloud-ML/store/models/cifar100_vgg.keras") #VGG16(weights='imagenet')
        return model

    def predict(self, X):
        X = self.preprocess(X)
        predictions = self.model.predict(X, verbose=None)
        return predictions

    def preprocess(self, X):
        
        X = X.copy()
        # X = (X * 10000).astype(np.uint8)

        if any(s < 32 for s in X.shape[1:3]):
            # Pad the input to make its size equal to 224
            padded_X = tf.image.resize_with_crop_or_pad(X, 32, 32)

            # Ensure the input is properly preprocessed for VGG16
            X = preprocess_input(padded_X.numpy())
        else:
            # If no padding is needed, directly preprocess the input
            X = preprocess_input(X)

        return X

    
    
cloud = VGG16CloudModel()


In [17]:
from tqdm import tqdm
X_encrypted, X_test_encrypted = [], []
X_embed, X_test_embed = [], []

with tf.device('/GPU:0'):
        
    for i, x in tqdm(X_sample.iterrows(), total=len(X_sample)):
        
        x_embed = embedding(x.values.reshape(1,-1))
        X_embed.append(x_embed)
        encrypted = encoder.encode(np.vstack(x_embed))
    
        X_encrypted.append(encrypted)
        
    for i,x in tqdm(X_test.iterrows(), total=len(X_test)):
        x_embed = embedding(x.values.reshape(1,-1))
        X_test_embed.append(x_embed)
        encrypted = encoder.encode(np.vstack(x_embed))
        X_test_encrypted.append(encrypted)

100%|██████████| 2000/2000 [00:11<00:00, 180.10it/s]


In [18]:
with tf.device('/GPU:0'):

    predictions = [
        cloud.predict(x)
        for x in tqdm(X_encrypted, total=len(X_encrypted))
    ]
    test_preds = [
        cloud.predict(x)
        for x in tqdm(X_test_encrypted, total=len(X_test_encrypted))
]

100%|██████████| 2000/2000 [00:39<00:00, 50.39it/s]


In [19]:

from keras.src.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, f1_score
from keras.src.models import Model
from keras.src.layers import Dense, Dropout, Input,  BatchNormalization
from keras.src.metrics import F1Score
import numpy as np

class NeuralNetworkInternalModel(BaseEstimator, ClassifierMixin):

    def __init__(self, **kwargs):
        self.batch_size = 8
        self.dropout_rate = 0.3
        self.epochs = 100
        self.model: Model = None

    def fit(self, X, y):
        y_onehot = to_categorical(y , 2)
        lr_scheduler = LearningRateScheduler(lambda epoch: 0.0001 * (0.9 ** epoch))
        early_stopping = EarlyStopping(patience=3, monitor='loss')
        self.model.fit(X, y_onehot, epochs=self.epochs, batch_size=self.batch_size, callbacks=[lr_scheduler, early_stopping])

    def predict(self, X):
        prediction = self.model.predict(X)
        return np.argmax(prediction, axis=1)


    def evaluate(self, X, y):
        if len(y.shape) == 2:
            y = np.argmax(y, axis=1)

        pred = self.predict(X)
        return accuracy_score(y, pred), f1_score(y, pred, average='weighted')



## Using one vector

In [24]:

class DenseInternalModel(NeuralNetworkInternalModel):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.name = "neural_network"
        num_classes = kwargs.get("num_classes")
        input_shape = kwargs.get("input_shape")
        self.model = self.get_model(num_classes=num_classes, input_shape=input_shape)

    def get_model(self, num_classes, input_shape):
        # Build the model
        inputs = Input(shape=(input_shape,))  # Dynamic input shape

        # Define the hidden layers
        x = BatchNormalization()(inputs)
        x = Dense(units=128, activation='leaky_relu')(x)
        x = Dropout(self.dropout_rate)(x)

        # Define the output layer
        outputs = Dense(units=num_classes, activation='softmax')(x)

        # Create the model
        model = Model(inputs=inputs, outputs=outputs)

        # Compile the model with F1 Score
        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy', F1Score()]
                      )

        return model
    

### Using Pred Vector

In [26]:
# Code to run on CPU
with tf.device('/CPU:0'):
    iim = DenseInternalModel(num_classes=2, input_shape=predictions[0].shape[1])
    iim.fit(np.vstack(predictions), y_sample)
    print("--------PERFORMANCE-------")
    print(iim.evaluate(np.vstack(test_preds), y_test))

Epoch 1/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 300us/step - accuracy: 0.9143 - f1_score: 0.4809 - loss: 0.3684 - learning_rate: 1.0000e-04
Epoch 2/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 298us/step - accuracy: 0.9160 - f1_score: 0.4781 - loss: 0.2904 - learning_rate: 9.0000e-05
Epoch 3/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 299us/step - accuracy: 0.9141 - f1_score: 0.4775 - loss: 0.2949 - learning_rate: 8.1000e-05
Epoch 4/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 300us/step - accuracy: 0.9172 - f1_score: 0.4784 - loss: 0.2866 - learning_rate: 7.2900e-05
Epoch 5/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 299us/step - accuracy: 0.9119 - f1_score: 0.4769 - loss: 0.2997 - learning_rate: 6.5610e-05
Epoch 6/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 301us/step - accuracy: 0.9170 - f1_score: 0.4784 - loss: 0.2871 - learning_rate: 5.9049e-05
Epoch 7/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 297us/step - accuracy: 0.9164 - f1_score: 0.4782 - loss: 0.2892 - learning_rate: 5.3144e-05
--------PERFORMANCE-------
63/63 ━

### Using Both Prediction and Embedding as One Vector

In [27]:
X_pred_embed, X_test_pred_embed = [], []
for embed, pred in zip(X_embed, predictions):
    X_pred_embed.append(np.hstack([embed, pred]))
    
for embed, pred in zip(X_test_embed, test_preds):
    X_test_pred_embed.append(np.hstack([embed, pred]))

In [29]:
with tf.device('/CPU:0'):
    
    iim = DenseInternalModel(num_classes=2, input_shape=X_pred_embed[0].shape[1])
    iim.fit(np.vstack(X_pred_embed), y_sample)
    print("--------PERFORMANCE-------")
    print(iim.evaluate(np.vstack(X_test_pred_embed), y_test))


Epoch 1/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 316us/step - accuracy: 0.8703 - f1_score: 0.4862 - loss: 0.3651 - learning_rate: 1.0000e-04
Epoch 2/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 343us/step - accuracy: 0.9161 - f1_score: 0.5385 - loss: 0.2570 - learning_rate: 9.0000e-05
Epoch 3/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 310us/step - accuracy: 0.9177 - f1_score: 0.5694 - loss: 0.2505 - learning_rate: 8.1000e-05
Epoch 4/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 313us/step - accuracy: 0.9192 - f1_score: 0.5730 - loss: 0.2511 - learning_rate: 7.2900e-05
Epoch 5/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 327us/step - accuracy: 0.9140 - f1_score: 0.5575 - loss: 0.2612 - learning_rate: 6.5610e-05
Epoch 6/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 335us/step - accuracy: 0.9168 - f1_score: 0.5688 - loss: 0.2555 - learning_rate: 5.9049e-05
Epoch 7/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 324us/step - accuracy: 0.9172 - f1_score: 0.5801 - loss: 0.2551 - learning_rate: 5.3144e-05
Epoch 8/100
2500/2500 ━━━━━━━━━━━━

## Baseline

In [30]:
with tf.device('/CPU:0'):
    baseline = DenseInternalModel(num_classes=2, input_shape=X_sample.shape[1])
    baseline.fit(X_sample, y_sample)
    print("--------PERFORMANCE-------")
    print(baseline.evaluate(X_test, y_test))

Epoch 1/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 291us/step - accuracy: 0.8383 - f1_score: 0.4963 - loss: 0.4083 - learning_rate: 1.0000e-04
Epoch 2/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 285us/step - accuracy: 0.9148 - f1_score: 0.5537 - loss: 0.2423 - learning_rate: 9.0000e-05
Epoch 3/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 286us/step - accuracy: 0.9130 - f1_score: 0.5730 - loss: 0.2423 - learning_rate: 8.1000e-05
Epoch 4/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 284us/step - accuracy: 0.9116 - f1_score: 0.5555 - loss: 0.2364 - learning_rate: 7.2900e-05
Epoch 5/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 288us/step - accuracy: 0.9077 - f1_score: 0.5506 - loss: 0.2454 - learning_rate: 6.5610e-05
Epoch 6/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 286us/step - accuracy: 0.9192 - f1_score: 0.5600 - loss: 0.2258 - learning_rate: 5.9049e-05
Epoch 7/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 288us/step - accuracy: 0.9154 - f1_score: 0.5740 - loss: 0.2347 - learning_rate: 5.3144e-05
Epoch 8/100
2500/2500 ━━━━━━━━━━━━

## Using Two vectors

In [31]:
from keras.src.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.metrics import accuracy_score, f1_score
from keras.src.models import Model
from keras.src.layers import Dense, Dropout, Input,  BatchNormalization, concatenate
from keras.src.metrics import F1Score
from keras.src import regularizers

class DoubleDenseInternalModel(NeuralNetworkInternalModel):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.name = "neural_network"
        num_classes = kwargs.get("num_classes")
        input_shape = kwargs.get("input_shape")
        self.model = self.get_model(num_classes=num_classes, input_shape=input_shape)

    def get_model(self, num_classes, input_shape):
        inputs_sub_networks = []

        input_shape_a, input_shape_b = input_shape
        input_a = Input(shape=(input_shape_a,))

        x = Dense(input_shape_a // 2, activation="relu", kernel_regularizer=regularizers.L2(0.1), bias_regularizer=regularizers.L2(0.01))(
            input_a)
        x = BatchNormalization(momentum=0.7)(x)
        x = Dropout(0.3)(x)
        # x = Dense(input_shape_a / 2, activation="relu")(x)
        x = Model(inputs=input_a, outputs=x)

        inputs_sub_networks.append(x)

        input_b = Input(shape=(input_shape_b,))
        # the second branch operates on the second input
        y = Dense(input_shape_b // 4, activation="relu", kernel_regularizer=regularizers.L2(0.1),  bias_regularizer=regularizers.L2(0.01))(
            input_b)
        y = BatchNormalization(momentum=0.7)(y)
        y = Dropout(0.3)(y)
        y = Model(inputs=input_b, outputs=y)

        inputs_sub_networks.append(y)

        combined = concatenate([k.output for k in inputs_sub_networks])

        m = Dense(num_classes, activation="softmax", kernel_regularizer=regularizers.L2(0.1),
                  bias_regularizer=regularizers.L2(0.1))(combined)

        model = Model(inputs=[k.input for k in inputs_sub_networks], outputs=m)
        # Compile the model with F1 Score
        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy', F1Score()]
                      )

        return model

In [33]:
X_double_pred_embed, X_double_test_pred_embed = [np.vstack(X_embed), np.vstack(predictions)], [np.vstack(X_test_embed), np.vstack(test_preds)]


In [34]:
with tf.device('/CPU:0'):
    double_iim = DoubleDenseInternalModel(num_classes=2, input_shape=(X_double_pred_embed[0].shape[1],X_double_pred_embed[1].shape[1]))
    double_iim.fit(X_double_pred_embed, y_sample)
    print("--------PERFORMANCE-------")
    print(double_iim.evaluate(X_double_test_pred_embed, y_test))



Epoch 1/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 339us/step - accuracy: 0.6645 - f1_score: 0.4346 - loss: 3.3812 - learning_rate: 1.0000e-04
Epoch 2/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 333us/step - accuracy: 0.9152 - f1_score: 0.4779 - loss: 0.6652 - learning_rate: 9.0000e-05
Epoch 3/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 331us/step - accuracy: 0.9122 - f1_score: 0.4771 - loss: 0.4519 - learning_rate: 8.1000e-05
Epoch 4/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 335us/step - accuracy: 0.9177 - f1_score: 0.4785 - loss: 0.3603 - learning_rate: 7.2900e-05
Epoch 5/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 336us/step - accuracy: 0.9181 - f1_score: 0.4786 - loss: 0.3259 - learning_rate: 6.5610e-05
Epoch 6/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 334us/step - accuracy: 0.9163 - f1_score: 0.4782 - loss: 0.3151 - learning_rate: 5.9049e-05
Epoch 7/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 331us/step - accuracy: 0.9117 - f1_score: 0.4769 - loss: 0.3145 - learning_rate: 5.3144e-05
Epoch 8/100
2500/2500 ━━━━━━━━━━━━

# Correlation

In [44]:
rows = []
for embed, pred, label in zip(X_embed, predictions, y_sample):
    rows.append(np.hstack([embed, pred, np.vstack([label])]))
df = pd.DataFrame(np.vstack(rows))
df.head()

,0,1,2,3,4,5,6,7,8,9,...,99,100,101,102,103,104,105,106,107,108
0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,2.766979e-07,5.367829e-09,...,0.000001,0.000005,5.753773e-07,1.384776e-09,2.619639e-07,0.000001,1.063482e-09,1.579175e-07,0.000266,0.0
1,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,2.766979e-07,5.367829e-09,...,0.000001,0.000005,5.753773e-07,1.384776e-09,2.619639e-07,0.000001,1.063482e-09,1.579175e-07,0.000266,0.0
2,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,2.766727e-07,5.367589e-09,...,0.000001,0.000005,5.753566e-07,1.384754e-09,2.619567e-07,0.000001,1.063456e-09,1.579101e-07,0.000266,0.0
3,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,1.0,2.766951e-07,5.367594e-09,...,0.000001,0.000005,5.753533e-07,1.384647e-09,2.619532e-07,0.000001,1.063356e-09,1.579191e-07,0.000266,0.0
4,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,-1.0,2.766727e-07,5.367589e-09,...,0.000001,0.000005,5.753566e-07,1.384754e-09,2.619567e-07,0.000001,1.063456e-09,1.579101e-07,0.000266,0.0


In [61]:
rows = []
for embed, pred, label in zip(X_test_embed, test_preds, y_test):
    rows.append(np.hstack([embed, pred, np.vstack([label])]))

test_df = pd.DataFrame(np.vstack(rows))
test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,99,100,101,102,103,104,105,106,107,108
0,-0.996705,-1.0,-1.0,-0.999537,-1.000000,-1.0,-1.0,1.0,2.766912e-07,5.367964e-09,...,0.000001,0.000005,5.753951e-07,1.384787e-09,2.619570e-07,0.000001,1.063490e-09,1.579187e-07,0.000266,0.0
1,-1.000000,-1.0,-1.0,-1.000000,1.000000,-1.0,-1.0,1.0,2.766979e-07,5.367829e-09,...,0.000001,0.000005,5.753773e-07,1.384776e-09,2.619639e-07,0.000001,1.063482e-09,1.579175e-07,0.000266,0.0
2,-0.999570,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,1.0,2.766907e-07,5.367943e-09,...,0.000001,0.000005,5.753918e-07,1.384787e-09,2.619565e-07,0.000001,1.063482e-09,1.579196e-07,0.000266,0.0
3,-1.000000,-1.0,-1.0,1.000000,-0.998985,-1.0,-1.0,-1.0,2.766720e-07,5.367792e-09,...,0.000001,0.000005,5.753728e-07,1.384619e-09,2.619551e-07,0.000001,1.063365e-09,1.579138e-07,0.000266,0.0
4,-1.000000,-1.0,-1.0,-1.000000,-0.999996,-1.0,-1.0,1.0,2.766909e-07,5.367933e-09,...,0.000001,0.000005,5.753923e-07,1.384779e-09,2.619559e-07,0.000001,1.063486e-09,1.579189e-07,0.000266,0.0


In [58]:
correlation_matrix = df.corr()

TOP = 10
target_correlation = correlation_matrix.iloc[-1].abs().sort_values(ascending=False)
top_target_corr = target_correlation[1:TOP]  # Exclude the target's correlation with itself

print(f"Top {TOP} feature correlations with the target:")
print(top_target_corr)

Top 10 feature correlations with the target:
6      0.003050
2      0.007783
4      0.013605
67     0.015070
11     0.016073
60     0.024851
102    0.026671
1      0.029054
0      0.035994
Name: 108, dtype: float64


In [57]:
TOP = 20

# Get the top 10 feature-feature correlations
feature_correlation = correlation_matrix.abs().unstack()
feature_correlation = feature_correlation[feature_correlation < 1.0]  # Remove self-correlations
top_feature_corr = feature_correlation.sort_values(ascending=False)[:TOP]

print(f"\nTop {TOP} feature-feature correlations:")
print(top_feature_corr)


Top 20 feature-feature correlations:
91   8      0.991189
8    91     0.991189
101  9      0.986578
9    101    0.986578
44   43     0.986427
43   44     0.986427
59   100    0.985324
100  59     0.985324
20   23     0.984703
23   20     0.984703
107  52     0.982269
52   107    0.982269
19   106    0.980771
106  19     0.980771
25   66     0.979824
66   25     0.979824
44   58     0.979658
58   44     0.979658
107  99     0.978215
99   107    0.978215
dtype: float64


# Dim Reduction

In [60]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Assuming your data is in a numpy array called 'X' with shape (n_samples, 1009)
# If not, you need to load and prepare your data first
X,y = df.iloc[:, :-1], df.iloc[:, -1]
# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=10)
X_reduced = pca.fit_transform(X_scaled)

# X_reduced now has shape (n_samples, 10)

# Print the explained variance ratio
print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Total explained variance:", sum(pca.explained_variance_ratio_))

Explained variance ratio: [0.53415906 0.25490196 0.06251872 0.03584684 0.02870274 0.01985813
 0.01562191 0.01151777 0.01084    0.00854644]
Total explained variance: 0.9825135639825718


In [68]:
X_test,_ = test_df.iloc[:, :-1], test_df.iloc[:, -1]
# Standardize the data
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Apply PCA
pca = PCA(n_components=10)
X_test_reduced = pca.fit_transform(X_test_scaled)

# X_reduced now has shape (n_samples, 10)

# Print the explained variance ratio
print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Total explained variance:", sum(pca.explained_variance_ratio_))

Explained variance ratio: [0.53593826 0.2474006  0.05817848 0.04273527 0.02496998 0.01911342
 0.01823509 0.0139761  0.01179614 0.01007248]
Total explained variance: 0.9824158305038713


In [69]:
with tf.device('/CPU:0'):
    
    iim = DenseInternalModel(num_classes=2, input_shape=X_reduced.shape[1])
    iim.fit(X_reduced, y_sample)
    print("--------PERFORMANCE-------")
    print(iim.evaluate(X_test_reduced, y_test))

Epoch 1/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 285us/step - accuracy: 0.8543 - f1_score: 0.5073 - loss: 0.4010 - learning_rate: 1.0000e-04
Epoch 2/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 277us/step - accuracy: 0.9151 - f1_score: 0.5256 - loss: 0.2679 - learning_rate: 9.0000e-05
Epoch 3/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 276us/step - accuracy: 0.9144 - f1_score: 0.5435 - loss: 0.2659 - learning_rate: 8.1000e-05
Epoch 4/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 276us/step - accuracy: 0.9178 - f1_score: 0.5436 - loss: 0.2547 - learning_rate: 7.2900e-05
Epoch 5/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 275us/step - accuracy: 0.9171 - f1_score: 0.5415 - loss: 0.2559 - learning_rate: 6.5610e-05
Epoch 6/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 274us/step - accuracy: 0.9220 - f1_score: 0.5675 - loss: 0.2468 - learning_rate: 5.9049e-05
Epoch 7/100
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 1s 275us/step - accuracy: 0.9136 - f1_score: 0.5582 - loss: 0.2631 - learning_rate: 5.3144e-05
Epoch 8/100
2500/2500 ━━━━━━━━━━━━

In [67]:
X_test_reduced.shape, y_test.shape

((20000, 10), (2000,))